In [1]:
import pandas as pd
from malid import config, helpers, logger
from malid.datamodels import GeneLocus

In [2]:
# Export positives and negatives together.

In [3]:
df = pd.read_csv(config.paths.base_data_dir / "CoV-AbDab_130623.filtered.tsv", sep="\t")
df["rownum"] = range(df.shape[0])
df

,CDRH3,j_gene,v_gene,VHorVHH,Binds to,Doesn't Bind to,Neutralising Vs,Not Neutralising Vs,Protein + Epitope,Origin,Status,cdr3_seq_aa_q_trim,cdr3_aa_sequence_trim_len,rownum
0,ARDLVVYGLDY,IGHJ4,IGHV3-53,EVQLVESGGGLIQPGGSLRLSCAASGFIVSRNYMNWVRQAPGKGLE...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,S; RBD,B-cells; SARS-CoV2_WT Convalescent Patients,Positive,ARDLVVYGLDY,11,0
1,AREFDLTKIIMVPPY,IGHJ4,IGHV3-11,QVQMVESGGGLVRPGGSLRLSCAASGFTFSDYYMSWIRQAPGKGLE...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Delta;S...,NaN,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Delta;S...,SARS-CoV2_Beta;SARS-CoV2_Omicron-BA1;SARS-CoV2...,S; RBD,B-cells; SARS-CoV2_WT Convalescent Patients,Positive,AREFDLTKIIMVPPY,15,1
2,ARDSSGWHWGVPFDY,IGHJ4,IGHV3-30,QVQLVESGGGVVQPGRSLRLSCASSGFTFSTYHMHWVRQPPGKGLE...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,S; RBD,B-cells; SARS-CoV2_WT Vaccinee (BBIBP-CoV),Positive,ARDSSGWHWGVPFDY,15,2
3,VRGSGGIHDAFDI,IGHJ3,IGHV3-53,EVQLVESGGGLIQPGGSLRLSCAVSGFTVSRMSWVRQAPGKGLECV...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,SARS-CoV2_Omicron-BA2.75.2;SARS-CoV2-Omicron-B...,S; RBD,B-cells; SARS-CoV2_WT Vaccinee (BBIBP-CoV),Positive,VRGSGGIHDAFDI,13,3
4,ARVDPRYEGFDY,IGHJ4,IGHV3-53,EVQLVESGGGLIQPGGSLRLSCAASEFIVSRNYMSWVRQAPGKGLE...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,SARS-CoV2_Omicron-BA1;SARS-CoV2_Omicron-BA2;SA...,S; RBD,B-cells; SARS-CoV2_WT Vaccinee (BBIBP-CoV),Positive,ARVDPRYEGFDY,12,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7124,ARIPLRLRSGGRVAFDI,IGHJ3,IGHV4-59,QVQLQESGPGLVKPSETLSLTCTVSDGSISPYFWSWIRQPPGEGLE...,SARS-CoV1,SARS-CoV2_WT;MERS-CoV,NaN,NaN,S; non-RBD,B-cells; SARS-CoV1 Human Patient,Negative,ARIPLRLRSGGRVAFDI,17,7124
7125,ARVQDIVIVPAVYGMDV,IGHJ6,IGHV1-69,QVQLVQSGAEVKKPGSSVKVSCKTSGGTFVNFNSYAISWVRQAPGQ...,SARS-CoV1,SARS-CoV2_WT;MERS-CoV,NaN,NaN,S; non-RBD,B-cells; SARS-CoV1 Human Patient,Negative,ARVQDIVIVPAVYGMDV,17,7125
7126,ARHPSSIFRGTVFTPYYFDY,IGHJ4,IGHV4-59,QVQLQESGPGLVKPSETLSLTCTVSGGSVSDTPFYWGWIRQPPGKG...,SARS-CoV1 (weak),229E;HKU1;NL63;OC43;SARS-CoV2_WT,NaN,NaN,S; Unk,B-cells; SARS-CoV1 Human Patient,Negative,ARHPSSIFRGTVFTPYYFDY,20,7126
7127,VTQRDNSRDYFPHYFHDMDV,IGHJ6,IGHV3-30,QAQLVESGGALVQPGRSLRLSCAASGFTFRNYAMHWVRQAPATGLQ...,SARS-CoV1,SARS-CoV2_WT,SARS-CoV1,SARS-CoV2_WT,S; RBD,B-cells; SARS-CoV1 Human Patient,Negative,VTQRDNSRDYFPHYFHDMDV,20,7127


In [4]:
df_parse = pd.concat(
    [
        pd.read_csv(fname, sep="\t")
        for fname in (config.paths.base_data_dir / "cov_abdab_fasta_split").glob(
            "*.parsed.tsv"
        )
    ],
    axis=0,
)
# extract fasta ID
df_parse[["specimen_label", "rownum"]] = df_parse["query_id"].str.split(
    "|", expand=True
)
df_parse["rownum"] = df_parse["rownum"].astype(int)
for specimen_label, grp in df_parse.groupby("specimen_label"):
    assert not grp["rownum"].duplicated().any()
df_parse

,type,query_id,v_gene,global_percent_identity,FR1-IMGT,CDR1-IMGT,FR2-IMGT,CDR2-IMGT,FR3-IMGT,CDR3-IMGT (germline),specimen_label,rownum
0,V,covabdab|0,IGHV3-53*01,93.821649,EVQLVESGGGLIQPGGSLRLSCAAS,GFIVSRNY,MNWVRQAPGKGLEWVAL,IYSGGST,FYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYC,AR,covabdab,0
1,V,covabdab|1,IGHV3-11*04,92.866327,QVQMVESGGGLVRPGGSLRLSCAAS,GFTFSDYY,MSWIRQAPGKGLEWISY,ISSSGSSM,NYADSVKGRFTISRDNAKNSLFLQMNSLRAEDTAVYYC,AR,covabdab,1
2,V,covabdab|2,IGHV3-30*06,90.808163,QVQLVESGGGVVQPGRSLRLSCASS,GFTFSTYH,MHWVRQPPGKGLEWVAF,ISYDGSNY,YYSDSVKGRFTISRDNSKNTVYLQMNSLRAEDTALYYC,AR,covabdab,2
3,V,covabdab|3,IGHV3-53*01,88.639175,EVQLVESGGGLIQPGGSLRLSCAVS,GFTVSR,MSWVRQAPGKGLECVSV,IYTGGNT,DYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTALYYC,VR,covabdab,3
4,V,covabdab|4,IGHV3-53*01,96.907216,EVQLVESGGGLIQPGGSLRLSCAAS,EFIVSRNY,MSWVRQAPGKGLEWVSV,IYSGGST,YYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYC,AR,covabdab,4
...,...,...,...,...,...,...,...,...,...,...,...,...
7124,V,covabdab|7124,IGHV4-59*01,88.638144,QVQLQESGPGLVKPSETLSLTCTVS,DGSISPYF,WSWIRQPPGEGLEWIGY,IYYDGNT,NYSPSLKSRATISLDTSKNQFSLILTSVTAADTAVYYC,AR,covabdab,7124
7125,V,covabdab|7125,IGHV1-69*09,87.120792,QVQLVQSGAEVKKPGSSVKVSCKTS,GGTFVNFNSYA,ISWVRQAPGQGLEWMGR,IIPILGVT,NYAQKFQGRVTITADKATRTVYMELSNLGSEDTAMYYC,AR,covabdab,7125
7126,V,covabdab|7126,IGHV4-39*01,91.915152,QVQLQESGPGLVKPSETLSLTCTVS,GGSVSDTPFY,WGWIRQPPGKGLEWIGS,MYYTGST,YYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYC,AR,covabdab,7126
7127,V,covabdab|7127,IGHV3-30*16,82.114737,QAQLVESGGALVQPGRSLRLSCAAS,GFTFRNYA,MHWVRQAPATGLQWLAV,ITSDGRNK,FYADSVKGRFTISREDSKNTLYLQMDSLRGEDTAVYYC,NaN,covabdab,7127


In [5]:
df["rownum"].describe()

count    7129.000000
mean     3564.000000
std      2058.109367
min         0.000000
25%      1782.000000
50%      3564.000000
75%      5346.000000
max      7128.000000
Name: rownum, dtype: float64

In [6]:
df_parse["rownum"].describe()

count    7129.000000
mean     3564.000000
std      2058.109367
min         0.000000
25%      1782.000000
50%      3564.000000
75%      5346.000000
max      7128.000000
Name: rownum, dtype: float64

In [7]:
# Merge our IgBlast output
# Our IgBlast gives some different V gene calls, and provides CDR1+2 sequences

In [8]:
orig_shape = df.shape
df = pd.merge(
    df.drop(columns=["v_gene"]),
    df_parse.rename(
        columns={
            "CDR1-IMGT": "cdr1_seq_aa_q_trim",
            "CDR2-IMGT": "cdr2_seq_aa_q_trim",
            "FR1-IMGT": "fr1_seq_aa_q_trim",
            "FR2-IMGT": "fr2_seq_aa_q_trim",
            "FR3-IMGT": "fr3_seq_aa_q_trim",
            "global_percent_identity": "v_mut",
        }
    )[
        [
            "v_gene",
            "rownum",
            "fr1_seq_aa_q_trim",
            "cdr1_seq_aa_q_trim",
            "fr2_seq_aa_q_trim",
            "cdr2_seq_aa_q_trim",
            "fr3_seq_aa_q_trim",
            "v_mut",
        ]
    ],
    left_on=["rownum"],
    right_on=["rownum"],
    how="inner",
    validate="1:1",
)
assert df.shape[0] == min(orig_shape[0], df_parse.shape[0])

In [9]:
# convert percent identity to mutation rate
# note: this is the *amino acid mutation rate*, not the nucleotide mutation rate as we compute in main pipeline
# however, this is as good as we can get here.
df["v_mut"] = 1.0 - df["v_mut"] / 100.0
df["v_mut"].describe()

count    7129.000000
mean        0.061718
std         0.046177
min         0.000000
25%         0.030490
50%         0.051224
75%         0.082691
max         0.520449
Name: v_mut, dtype: float64

In [10]:
df.shape[0], orig_shape[0], df_parse.shape[0]

(7129, 7129, 7129)

In [11]:
df.head()

,CDRH3,j_gene,VHorVHH,Binds to,Doesn't Bind to,Neutralising Vs,Not Neutralising Vs,Protein + Epitope,Origin,Status,cdr3_seq_aa_q_trim,cdr3_aa_sequence_trim_len,rownum,v_gene,fr1_seq_aa_q_trim,cdr1_seq_aa_q_trim,fr2_seq_aa_q_trim,cdr2_seq_aa_q_trim,fr3_seq_aa_q_trim,v_mut
0,ARDLVVYGLDY,IGHJ4,EVQLVESGGGLIQPGGSLRLSCAASGFIVSRNYMNWVRQAPGKGLE...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,S; RBD,B-cells; SARS-CoV2_WT Convalescent Patients,Positive,ARDLVVYGLDY,11,0,IGHV3-53*01,EVQLVESGGGLIQPGGSLRLSCAAS,GFIVSRNY,MNWVRQAPGKGLEWVAL,IYSGGST,FYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYC,0.061784
1,AREFDLTKIIMVPPY,IGHJ4,QVQMVESGGGLVRPGGSLRLSCAASGFTFSDYYMSWIRQAPGKGLE...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Delta;S...,NaN,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Delta;S...,SARS-CoV2_Beta;SARS-CoV2_Omicron-BA1;SARS-CoV2...,S; RBD,B-cells; SARS-CoV2_WT Convalescent Patients,Positive,AREFDLTKIIMVPPY,15,1,IGHV3-11*04,QVQMVESGGGLVRPGGSLRLSCAAS,GFTFSDYY,MSWIRQAPGKGLEWISY,ISSSGSSM,NYADSVKGRFTISRDNAKNSLFLQMNSLRAEDTAVYYC,0.071337
2,ARDSSGWHWGVPFDY,IGHJ4,QVQLVESGGGVVQPGRSLRLSCASSGFTFSTYHMHWVRQPPGKGLE...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,S; RBD,B-cells; SARS-CoV2_WT Vaccinee (BBIBP-CoV),Positive,ARDSSGWHWGVPFDY,15,2,IGHV3-30*06,QVQLVESGGGVVQPGRSLRLSCASS,GFTFSTYH,MHWVRQPPGKGLEWVAF,ISYDGSNY,YYSDSVKGRFTISRDNSKNTVYLQMNSLRAEDTALYYC,0.091918
3,VRGSGGIHDAFDI,IGHJ3,EVQLVESGGGLIQPGGSLRLSCAVSGFTVSRMSWVRQAPGKGLECV...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,SARS-CoV2_Omicron-BA2.75.2;SARS-CoV2-Omicron-B...,S; RBD,B-cells; SARS-CoV2_WT Vaccinee (BBIBP-CoV),Positive,VRGSGGIHDAFDI,13,3,IGHV3-53*01,EVQLVESGGGLIQPGGSLRLSCAVS,GFTVSR,MSWVRQAPGKGLECVSV,IYTGGNT,DYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTALYYC,0.113608
4,ARVDPRYEGFDY,IGHJ4,EVQLVESGGGLIQPGGSLRLSCAASEFIVSRNYMSWVRQAPGKGLE...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,SARS-CoV2_Omicron-BA1;SARS-CoV2_Omicron-BA2;SA...,S; RBD,B-cells; SARS-CoV2_WT Vaccinee (BBIBP-CoV),Positive,ARVDPRYEGFDY,12,4,IGHV3-53*01,EVQLVESGGGLIQPGGSLRLSCAAS,EFIVSRNY,MSWVRQAPGKGLEWVSV,IYSGGST,YYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYC,0.030928


In [12]:
# sanity check
assert df["v_gene"].str.startswith("IGHV").all()
assert (df["cdr1_seq_aa_q_trim"].str.strip() == df["cdr1_seq_aa_q_trim"]).all()
assert (df["cdr2_seq_aa_q_trim"].str.strip() == df["cdr2_seq_aa_q_trim"]).all()

In [13]:
df["v_gene"].value_counts()

IGHV3-30*16    497
IGHV3-30*18    381
IGHV1-69*13    332
IGHV3-53*01    290
IGHV3-9*01     286
              ... 
IGHV2-70*12      1
IGHV3-9*02       1
IGHV3-15*04      1
IGHV4-59*05      1
IGHV1-2*03       1
Name: v_gene, Length: 128, dtype: int64

In [14]:
df["v_gene"] = df["v_gene"].str.split("*").str[0].astype("category")

In [15]:
df["v_gene"].value_counts()

IGHV3-30        1078
IGHV1-69         717
IGHV3-53         376
IGHV3-33         332
IGHV3-23         330
IGHV3-9          288
IGHV4-39         286
IGHV1-46         285
IGHV5-51         259
IGHV1-18         204
IGHV3-66         203
IGHV4-4          185
IGHV4-59         178
IGHV1-2          178
IGHV3-13         163
IGHV4-31         156
IGHV3-7          145
IGHV3-21         144
IGHV3-48         132
IGHV2-5          131
IGHV4-34         131
IGHV1-58          98
IGHV3-15          93
IGHV2-70          90
IGHV3-30-3        82
IGHV1-24          82
IGHV7-4-1         79
IGHV3-11          79
IGHV4-61          73
IGHV1-8           72
IGHV4-30-4        57
IGHV5-a           56
IGHV1-3           52
IGHV3-64D         46
IGHV3-49          43
IGHV3-20          36
IGHV3-74          28
IGHV4-b           28
IGHV3-43          26
IGHV4-30-2        19
IGHV2-26          19
IGHV6-1           17
IGHV2-70D         10
IGHV3-64           9
IGHV1-f            8
IGHV3-73           7
IGHV3-43D          7
IGHV3-72     

In [16]:
# Note: we don't know isotype
# TODO: can we subset to matches in our dataset, and get the true isotype?

In [17]:
# Downselect only to V genes that are in our data
# We will never get matches on the rest. Clustering will always fail.
invalid_v_genes = set(df["v_gene"].unique()) - set(
    helpers.all_observed_v_genes()[GeneLocus.BCR]
)
logger.warning(f"Dropping CoV-AbDab V genes that aren't in our data: {invalid_v_genes}")
df = df[df["v_gene"].isin(helpers.all_observed_v_genes()[GeneLocus.BCR])]

{"message": "Dropping CoV-AbDab V genes that aren't in our data: set()", "time": "2023-11-08T20:19:06.738849"}


# Export

In [18]:
df

,CDRH3,j_gene,VHorVHH,Binds to,Doesn't Bind to,Neutralising Vs,Not Neutralising Vs,Protein + Epitope,Origin,Status,cdr3_seq_aa_q_trim,cdr3_aa_sequence_trim_len,rownum,v_gene,fr1_seq_aa_q_trim,cdr1_seq_aa_q_trim,fr2_seq_aa_q_trim,cdr2_seq_aa_q_trim,fr3_seq_aa_q_trim,v_mut
0,ARDLVVYGLDY,IGHJ4,EVQLVESGGGLIQPGGSLRLSCAASGFIVSRNYMNWVRQAPGKGLE...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,S; RBD,B-cells; SARS-CoV2_WT Convalescent Patients,Positive,ARDLVVYGLDY,11,0,IGHV3-53,EVQLVESGGGLIQPGGSLRLSCAAS,GFIVSRNY,MNWVRQAPGKGLEWVAL,IYSGGST,FYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYC,0.061784
1,AREFDLTKIIMVPPY,IGHJ4,QVQMVESGGGLVRPGGSLRLSCAASGFTFSDYYMSWIRQAPGKGLE...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Delta;S...,NaN,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Delta;S...,SARS-CoV2_Beta;SARS-CoV2_Omicron-BA1;SARS-CoV2...,S; RBD,B-cells; SARS-CoV2_WT Convalescent Patients,Positive,AREFDLTKIIMVPPY,15,1,IGHV3-11,QVQMVESGGGLVRPGGSLRLSCAAS,GFTFSDYY,MSWIRQAPGKGLEWISY,ISSSGSSM,NYADSVKGRFTISRDNAKNSLFLQMNSLRAEDTAVYYC,0.071337
2,ARDSSGWHWGVPFDY,IGHJ4,QVQLVESGGGVVQPGRSLRLSCASSGFTFSTYHMHWVRQPPGKGLE...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,S; RBD,B-cells; SARS-CoV2_WT Vaccinee (BBIBP-CoV),Positive,ARDSSGWHWGVPFDY,15,2,IGHV3-30,QVQLVESGGGVVQPGRSLRLSCASS,GFTFSTYH,MHWVRQPPGKGLEWVAF,ISYDGSNY,YYSDSVKGRFTISRDNSKNTVYLQMNSLRAEDTALYYC,0.091918
3,VRGSGGIHDAFDI,IGHJ3,EVQLVESGGGLIQPGGSLRLSCAVSGFTVSRMSWVRQAPGKGLECV...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,SARS-CoV2_Omicron-BA2.75.2;SARS-CoV2-Omicron-B...,S; RBD,B-cells; SARS-CoV2_WT Vaccinee (BBIBP-CoV),Positive,VRGSGGIHDAFDI,13,3,IGHV3-53,EVQLVESGGGLIQPGGSLRLSCAVS,GFTVSR,MSWVRQAPGKGLECVSV,IYTGGNT,DYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTALYYC,0.113608
4,ARVDPRYEGFDY,IGHJ4,EVQLVESGGGLIQPGGSLRLSCAASEFIVSRNYMSWVRQAPGKGLE...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,SARS-CoV2_Omicron-BA1;SARS-CoV2_Omicron-BA2;SA...,S; RBD,B-cells; SARS-CoV2_WT Vaccinee (BBIBP-CoV),Positive,ARVDPRYEGFDY,12,4,IGHV3-53,EVQLVESGGGLIQPGGSLRLSCAAS,EFIVSRNY,MSWVRQAPGKGLEWVSV,IYSGGST,YYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYC,0.030928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7124,ARIPLRLRSGGRVAFDI,IGHJ3,QVQLQESGPGLVKPSETLSLTCTVSDGSISPYFWSWIRQPPGEGLE...,SARS-CoV1,SARS-CoV2_WT;MERS-CoV,NaN,NaN,S; non-RBD,B-cells; SARS-CoV1 Human Patient,Negative,ARIPLRLRSGGRVAFDI,17,7124,IGHV4-59,QVQLQESGPGLVKPSETLSLTCTVS,DGSISPYF,WSWIRQPPGEGLEWIGY,IYYDGNT,NYSPSLKSRATISLDTSKNQFSLILTSVTAADTAVYYC,0.113619
7125,ARVQDIVIVPAVYGMDV,IGHJ6,QVQLVQSGAEVKKPGSSVKVSCKTSGGTFVNFNSYAISWVRQAPGQ...,SARS-CoV1,SARS-CoV2_WT;MERS-CoV,NaN,NaN,S; non-RBD,B-cells; SARS-CoV1 Human Patient,Negative,ARVQDIVIVPAVYGMDV,17,7125,IGHV1-69,QVQLVQSGAEVKKPGSSVKVSCKTS,GGTFVNFNSYA,ISWVRQAPGQGLEWMGR,IIPILGVT,NYAQKFQGRVTITADKATRTVYMELSNLGSEDTAMYYC,0.128792
7126,ARHPSSIFRGTVFTPYYFDY,IGHJ4,QVQLQESGPGLVKPSETLSLTCTVSGGSVSDTPFYWGWIRQPPGKG...,SARS-CoV1 (weak),229E;HKU1;NL63;OC43;SARS-CoV2_WT,NaN,NaN,S; Unk,B-cells; SARS-CoV1 Human Patient,Negative,ARHPSSIFRGTVFTPYYFDY,20,7126,IGHV4-39,QVQLQESGPGLVKPSETLSLTCTVS,GGSVSDTPFY,WGWIRQPPGKGLEWIGS,MYYTGST,YYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYC,0.080848
7127,VTQRDNSRDYFPHYFHDMDV,IGHJ6,QAQLVESGGALVQPGRSLRLSCAASGFTFRNYAMHWVRQAPATGLQ...,SARS-CoV1,SARS-CoV2_WT,SARS-CoV1,SARS-CoV2_WT,S; RBD,B-cells; SARS-CoV1 Human Patient,Negative,VTQRDNSRDYFPHYFHDMDV,20,7127,IGHV3-30,QAQLVESGGALVQPGRSLRLSCAAS,GFTFRNYA,MHWVRQAPATGLQWLAV,ITSDGRNK,FYADSVKGRFTISREDSKNTLYLQMDSLRGEDTAVYYC,0.178853


In [19]:
concatenated_parts_available_to_us = (
    df["fr1_seq_aa_q_trim"]
    + df["cdr1_seq_aa_q_trim"]
    + df["fr2_seq_aa_q_trim"]
    + df["cdr2_seq_aa_q_trim"]
    + df["fr3_seq_aa_q_trim"]
    + df["cdr3_seq_aa_q_trim"]
)

In [20]:
df.iloc[0]["VHorVHH"]

'EVQLVESGGGLIQPGGSLRLSCAASGFIVSRNYMNWVRQAPGKGLEWVALIYSGGSTFYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCARDLVVYGLDYWGQGTLVTVSS'

In [21]:
concatenated_parts_available_to_us.iloc[0]

'EVQLVESGGGLIQPGGSLRLSCAASGFIVSRNYMNWVRQAPGKGLEWVALIYSGGSTFYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCARDLVVYGLDY'

In [22]:
# Create FR4 field that is VHorVHH minus the concatenation (we verified this with IgBlast manually)
# Basically igblastp does not give us the FR4 region.
# But for our modeling purposes, we can use VHorVHH as the concatenated string.

In [23]:
df["post_seq_aa_q_trim"] = [
    fullstring.replace(substring, "").strip()
    for fullstring, substring in zip(df["VHorVHH"], concatenated_parts_available_to_us)
]
df["post_seq_aa_q_trim"].head()

0    WGQGTLVTVSS
1    WGQGTLVTVSS
2    WGQGTLVTVSS
3    WGQGTMVTVSS
4    WGQGTLVTVSS
Name: post_seq_aa_q_trim, dtype: object

In [24]:
# However this approach can sometimes fail, if there's a difference between VHorVHH and the IgBlast fields up to FR3 + CDR3 field in IgBlast
# Below is an example where we see this failure. It looks like the difference is in the CDR3 region, which is provided by CoV-AbDab. Not sure why they have a discrepancy between their own fields.
# Reviewing this particular example in the original CoV-AbDab spreadsheet, we see that the VHorVHH sequence appears several times, and each row has a different CDRH3 annotation. One of the rows has a match between VHorVHH and CDRH3, so it is not flagged.
# We are safe to drop the flagged rows.

In [25]:
failed_subtractions = df["post_seq_aa_q_trim"] == df["VHorVHH"]
failed_subtractions.value_counts()

False    7103
True       26
dtype: int64

In [26]:
failed_subtractions

0       False
1       False
2       False
3       False
4       False
        ...  
7124    False
7125    False
7126    False
7127    False
7128    False
Length: 7129, dtype: bool

In [27]:
df[failed_subtractions]["post_seq_aa_q_trim"]

237     EVQLVESGGGLGQPGGSLRLSCVASGFSFSTYSMNWVRQAPGKGLE...
4604    EVQLVESGGGLVQPGGSLRLSCSASGFTFSSYAMHWVRQAPGKGLE...
4790    QVQLVQSGAEVKKPGSSVKVSCKASGDTFTNYAFSWMRQAPGQGLE...
5187    EVQLVESGAEVKKPGSSVKVSCKASGGTFSSYAISWVRQAPGQGLE...
5192    EVQLVESGGGVVQPGRSLRLSCAASGFTFSSYTMHWVRQAPGKGLE...
5193    EVQLVESGPGLVKPSQTLSLTCTVSGGSISSGDYYWSWIRQPPGKG...
5212    QVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...
5213    EVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...
5545    QVQLVQSGAEVKKPGSSVKVSCKASGGTFSIYAITWVRQAPGQGLE...
5546    QVQLVQSGAEVKKPGSSVKVSCKASGGTFAIYAITWVRQAPGQGLE...
5547    QVQLVQSGAEVKKPGSSVKVSCKASGGAFSIYAITWVRQAPGQGLE...
5548    QVQLVQSGAEVKKPGSSVKVSCKASGGAFSIYSITWVRQAPGQGLE...
5549    QVQLVQSGAEVKKPGSSVKVSCKVSGGWFSIYAITWVRQAPGQGLE...
5550    QVQLVQSGAEVKKPGSSVKVSCKASGGAFSIYAITWVRQAPGQGLE...
5551    QVQLVQSGAEVKKPGSSVKVSCKASGGTFSIYALTWVRQAPGQGLE...
5552    QVQLVQSGAEVKKPGSSVKVSCKASGGTFSIYAITWVRQAPGQGLE...
5553    QVQLVQSGAEVKKPGSSVKVSCKASGGIFSIYAITWVRQAPGQGLE...
5554    QVQLVQ

In [28]:
concatenated_parts_available_to_us.loc[failed_subtractions].iloc[0]

'EVQLVESGGGLGQPGGSLRLSCVASGFSFSTYSMNWVRQAPGKGLEWLSYINAGSTTIYSADSVTGRFSISRDNAKNSLYLHMNSLRADDTAVYYCSSYTSRIPII'

In [29]:
df.loc[failed_subtractions].iloc[0]["VHorVHH"]

'EVQLVESGGGLGQPGGSLRLSCVASGFSFSTYSMNWVRQAPGKGLEWLSYINAGSTTIYSADSVTGRFSISRDNAKNSLYLHMNSLRADDTAVYYCARGVPFDYWGQGTLVTVSS'

In [30]:
df.loc[failed_subtractions].iloc[0]["CDRH3"]

'SSYTSRIPII'

In [31]:
# Drop the flagged rows.
df = df.loc[~failed_subtractions]

In [32]:
assert not (df["post_seq_aa_q_trim"] == df["VHorVHH"]).any()

In [33]:
df["post_seq_aa_q_trim"].head()

0    WGQGTLVTVSS
1    WGQGTLVTVSS
2    WGQGTLVTVSS
3    WGQGTMVTVSS
4    WGQGTLVTVSS
Name: post_seq_aa_q_trim, dtype: object

In [34]:
df["post_seq_aa_q_trim"].value_counts()

WGQGTLVTVSS    3497
WGQGTTVTVSS    1273
WGQGTMVTVSS     764
WGRGTLVTVSS     227
WGKGTTVTVSS     215
               ... 
TRTTCSDGFDI       1
WGQGTLLSVSS       1
WGQGTTLTVSS       1
WGQGTLVSVS        1
WGQGALVTVSA       1
Name: post_seq_aa_q_trim, Length: 319, dtype: int64

In [35]:
df.columns

Index(['CDRH3', 'j_gene', 'VHorVHH', 'Binds to', 'Doesn't Bind to',
       'Neutralising Vs', 'Not Neutralising Vs', 'Protein + Epitope', 'Origin',
       'Status', 'cdr3_seq_aa_q_trim', 'cdr3_aa_sequence_trim_len', 'rownum',
       'v_gene', 'fr1_seq_aa_q_trim', 'cdr1_seq_aa_q_trim',
       'fr2_seq_aa_q_trim', 'cdr2_seq_aa_q_trim', 'fr3_seq_aa_q_trim', 'v_mut',
       'post_seq_aa_q_trim'],
      dtype='object')

In [36]:
df["Status"].value_counts()

Positive    6517
Negative     586
Name: Status, dtype: int64

In [37]:
assert not df["Status"].isna().any()

In [38]:
df.drop(columns="rownum").to_csv(
    config.paths.base_data_dir / "CoV-AbDab_130623.filtered.annotated.tsv",
    sep="\t",
    index=None,
)